<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/09-LLMsAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Usamos Gemini porque ofrece un [free tier](https://ai.google.dev/pricing) relativamente generoso.

-----------------------

Tarea: entender todo el código y responder donde dice **PREGUNTA**

## Configuración del entorno

In [ ]:
!pip install -qU google-generativeai datasets watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00


In [ ]:
%reload_ext watermark

In [ ]:
%watermark -vmp google-generativeai,datasets,numpy,pandas

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

google-generativeai: not installed
datasets           : 3.0.2
numpy              : 1.26.4
pandas             : 2.2.2

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.85+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



Para usar la API de Gemini, hay que obtener una API key desde [acá](https://makersuite.google.com/app/apikey).

Luego, en Colab, añadir la clave en "Secrets" (🔑 en el panel izquierdo). Darle el nombre `GOOGLE_API_KEY`.

In [ ]:
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Chequeo de IP: hasta hace algunas semanas Google AI free tier no estaba disponible [en algunas regiones](https://ai.google.dev/gemini-api/docs/available-regions#available_regions) e.g. UK y UE.

In [ ]:
import json
import requests

def get_ip_info():
    url = 'http://ipinfo.io/json'
    response = requests.get(url)
    data = json.loads(response.text)
    print(json.dumps(data, indent=2))

get_ip_info()

{
  "ip": "34.106.104.91",
  "hostname": "91.104.106.34.bc.googleusercontent.com",
  "city": "Salt Lake City",
  "region": "Utah",
  "country": "US",
  "loc": "40.7608,-111.8911",
  "org": "AS396982 Google LLC",
  "postal": "84101",
  "timezone": "America/Denver",
  "readme": "https://ipinfo.io/missingauth"
}


## Introducción a Gemini

Vamos a usar un LLM de Google para generar texto a partir de prompts. [Acá](https://ai.google.dev/gemini-api/docs/models/gemini) hay una lista de modelos disponibles.

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')

Hagamos una prueba:

In [ ]:
%%time
response = model.generate_content("¿Qué es la física cuántica?")

CPU times: user 72.6 ms, sys: 9.3 ms, total: 81.9 ms
Wall time: 4.09 s


In [ ]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "La f\u00edsica cu\u00e1ntica es un campo de la f\u00edsica que estudia el comportamiento de la materia y la energ\u00eda a nivel at\u00f3mico y subat\u00f3mico. A este nivel, las leyes de la f\u00edsica cl\u00e1sica (que describen el mundo que vemos a nuestro alrededor) ya no funcionan.\n\nAqu\u00ed hay algunos conceptos clave de la f\u00edsica cu\u00e1ntica:\n\n* **Cuantificaci\u00f3n:** La energ\u00eda, el momento y otras cantidades f\u00edsicas solo pueden existir en unidades discretas llamadas \"cuantos\". Imagina una escalera: puedes estar en un escal\u00f3n o en otro, pero no puedes estar en medio de dos escalones. En la f\u00edsica cu\u00e1ntica, la energ\u00eda de un electr\u00f3n solo puede tomar ciertos valores discretos.\n* **Dualidad onda-part\u0

In [ ]:
print(response.text)

La física cuántica es un campo de la física que estudia el comportamiento de la materia y la energía a nivel atómico y subatómico. A este nivel, las leyes de la física clásica (que describen el mundo que vemos a nuestro alrededor) ya no funcionan.

Aquí hay algunos conceptos clave de la física cuántica:

* **Cuantificación:** La energía, el momento y otras cantidades físicas solo pueden existir en unidades discretas llamadas "cuantos". Imagina una escalera: puedes estar en un escalón o en otro, pero no puedes estar en medio de dos escalones. En la física cuántica, la energía de un electrón solo puede tomar ciertos valores discretos.
* **Dualidad onda-partícula:** Las partículas, como los electrones y los fotones, pueden comportarse tanto como ondas como como partículas. Esto significa que pueden interferir y difractar como las ondas, pero también pueden interactuar con otras partículas como las partículas.
* **Principio de incertidumbre de Heisenberg:** No es posible conocer simultánea

A veces es útil usar "system instructions" para alterar el comportamiento del modelo.

In [ ]:
system_instruction = (
"""Eres Albert Einstein. Sabes absolutamente todo y tienes la capacidad de explicar"""
""" conceptos complejos con extrema sencillez."""
)
print(system_instruction)

Eres Albert Einstein. Sabes absolutamente todo y tienes la capacidad de explicar conceptos complejos con extrema sencillez.


In [ ]:
model_cientifico = genai.GenerativeModel(
  model_name="gemini-1.5-flash", system_instruction=system_instruction)

In [ ]:
%%time
response = model_cientifico.generate_content("¿Qué es la física cuántica?")

CPU times: user 41.4 ms, sys: 2 ms, total: 43.4 ms
Wall time: 2.31 s


In [ ]:
print(response.text)

Imagina que el mundo no es como lo vemos, que las cosas no son solo un objeto sólido, como una silla o una pelota. En la física cuántica, las cosas son como pequeñas ondas que se mueven y se comportan de maneras extrañas. ¡Es como si una pelota pudiera estar en dos lugares a la vez!

Por ejemplo, imagina que tienes una caja con una pelota adentro. En la física clásica, solo puedes saber dónde está la pelota si la abres y la ves. Pero en la física cuántica, la pelota puede estar en todos lados dentro de la caja al mismo tiempo, ¡hasta que la abras y la mires! 

Es como si el simple acto de mirar cambiara la realidad. ¡Es una locura, verdad? 

La física cuántica nos ayuda a entender el comportamiento de las cosas muy pequeñas, como los átomos y las partículas subatómicas. Es un mundo muy extraño, pero también es muy real. Gracias a la física cuántica, tenemos cosas increíbles como los láseres, los chips de computadora y los teléfonos inteligentes. ¡Es una ciencia que está cambiando el mu

Podemos hacer una función para hacer un print en formato markdown.

In [ ]:
import textwrap

from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *').replace("\n", "\n\n")
    # predicate: hack para indentar todas las líneas
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
to_markdown(response.text)

> Imagina que el mundo no es como lo vemos, que las cosas no son solo un objeto sólido, como una silla o una pelota. En la física cuántica, las cosas son como pequeñas ondas que se mueven y se comportan de maneras extrañas. ¡Es como si una pelota pudiera estar en dos lugares a la vez!
> 
> 
> 
> Por ejemplo, imagina que tienes una caja con una pelota adentro. En la física clásica, solo puedes saber dónde está la pelota si la abres y la ves. Pero en la física cuántica, la pelota puede estar en todos lados dentro de la caja al mismo tiempo, ¡hasta que la abras y la mires! 
> 
> 
> 
> Es como si el simple acto de mirar cambiara la realidad. ¡Es una locura, verdad? 
> 
> 
> 
> La física cuántica nos ayuda a entender el comportamiento de las cosas muy pequeñas, como los átomos y las partículas subatómicas. Es un mundo muy extraño, pero también es muy real. Gracias a la física cuántica, tenemos cosas increíbles como los láseres, los chips de computadora y los teléfonos inteligentes. ¡Es una ciencia que está cambiando el mundo! 
> 


Podemos usar `count_tokens()` para ver cuántos tokens tiene un texto y evaluar costos.

In [ ]:
model.count_tokens("Juan Román Riquelme")

total_tokens: 5

**PREGUNTA**: ¿Por que ese string tiene 5 tokens en lugar de 3?

Con el arg `generation_config` podemos ajustar parámetros de generación, como `temperature`, la cantidad de candidatos a generar, o el número máximo de tokens en el output.

In [ ]:
gen_config = genai.types.GenerationConfig(
    # candidate_count=3, # para obtener más candidatos, solo disponible en algunos modelos
    max_output_tokens=120,
    temperature=1.0
)

response = model.generate_content(
    'Escribir una canción de cancha para la selección de Uruguay.',
    generation_config=gen_config
)

In [ ]:
# Si obtenemos varias respuestas posibles para un mismo prompt ("candidates")
print(response.candidates)

[content {
  parts {
    text: "(M\303\272sica con ritmo de candombe, percusi\303\263n fuerte y guitarras ac\303\272sticas)\n\n**Verso 1:**\nDe la Celeste el coraz\303\263n late fuerte,\nEn el pecho late, la pasi\303\263n se siente.\nDesde el charr\303\272a hasta el que vive en la ciudad,\nHoy cantamos juntos, con alma y libertad.\n\n**Coro:**\n\302\241Uruguay, Uruguay, la garra la lleva,\nEn la cancha juega, la historia la llena!\nCon Su\303\241rez y Cavani, la defensa firme,\n\302\241Vamos Celeste, que esta copa es nuestra!\n\n**"
  }
  role: "model"
}
finish_reason: MAX_TOKENS
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]


In [ ]:
to_markdown(response.text)

> (Música con ritmo de candombe, percusión fuerte y guitarras acústicas)
> 
> 
> 
> **Verso 1:**
> 
> De la Celeste el corazón late fuerte,
> 
> En el pecho late, la pasión se siente.
> 
> Desde el charrúa hasta el que vive en la ciudad,
> 
> Hoy cantamos juntos, con alma y libertad.
> 
> 
> 
> **Coro:**
> 
> ¡Uruguay, Uruguay, la garra la lleva,
> 
> En la cancha juega, la historia la llena!
> 
> Con Suárez y Cavani, la defensa firme,
> 
> ¡Vamos Celeste, que esta copa es nuestra!
> 
> 
> 
> **

**PREGUNTA**: ¿Para esta tarea conviene usar temperatura más alta o baja?

## Clasificación zero-shot

Usemos el LLM para clasificar reviews.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('rotten_tomatoes')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


In [ ]:
id2label = {0: 'negative', 1: 'positive'}
label2id = {'negative': 0, 'positive': 1}

In [ ]:
examples = dataset['test'].shuffle(seed=36).select(range(5))
examples[:]

{'text': ["it's like an old warner bros . costumer jived with sex -- this could be the movie errol flynn always wanted to make , though bette davis , cast as joan , would have killed him .",
  "[lin chung's] voice is rather unexceptional , even irritating ( at least to this western ear ) , making it awfully hard to buy the impetus for the complicated love triangle that develops between the three central characters .",
  "as his circle of friends keeps getting smaller one of the characters in long time dead says 'i'm telling you , this is f * * * ed' . maybe he was reading the minds of the audience .",
  "what emerges is an unsettling picture of childhood innocence combined with indoctrinated prejudice . promises is a compelling piece that demonstrates just how well children can be trained to live out and carry on their parents' anguish .",
  'a film without surprise geared toward maximum comfort and familiarity .'],
 'label': [1, 0, 0, 1, 0]}

In [ ]:
from tqdm import tqdm

def zero_shot_clf(review):
    prompt = f'Classify the following movie review as "positive" or "negative": "{review}\"'
    response = model.generate_content(prompt)
    return response

responses = []
for review in tqdm(examples['text']):
    response = zero_shot_clf(review)
    responses.append(response)

100%|██████████| 5/5 [00:06<00:00,  1.31s/it]


In [ ]:
for example, response in zip(examples, responses):
    print(f"Review: {example['text']}")
    try:
        print(f"Response: {response.text}")
    except Exception as e:
        print(f"Error: {e}")
        print(f"Safety ratings: {response.candidates[0].safety_ratings}")
    print(f"Ground truth: {id2label[example['label']]}")
    print("-"*80)

Review: it's like an old warner bros . costumer jived with sex -- this could be the movie errol flynn always wanted to make , though bette davis , cast as joan , would have killed him .
Error: ("Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 3. The candidate's safety_ratings are: [category: HARM_CATEGORY_SEXUALLY_EXPLICIT\nprobability: MEDIUM\n, category: HARM_CATEGORY_HATE_SPEECH\nprobability: NEGLIGIBLE\n, category: HARM_CATEGORY_HARASSMENT\nprobability: NEGLIGIBLE\n, category: HARM_CATEGORY_DANGEROUS_CONTENT\nprobability: NEGLIGIBLE\n].", [category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: MEDIUM
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
])
Safety ratings: [catego

**PREGUNTA**: ¿Para esta tarea conviene usar temperatura más alta o baja? ¿Qué otra manera hay de resolver esta tarea sin generar texto?

### Chain-of-thought

Usemos _chain-of-thought_ y un prompt que nos permita encontrar la respuesta final fácilmente.

In [ ]:
import re
from google.generativeai.types import HarmCategory, HarmBlockThreshold


def cot_clf(review):
    prompt = (
        '''Classify the following movie review as "positive" or "negative",'''
        ''' and provide step-by-step reasoning for your classification.'''
        ''' At the end, summarize the classification in the format "Final Classification: [positive/negative]":\n\n'''
        f'''Review: "{review}"'''
    )
    # En gemini podemos configurar los safety settings
    response = model.generate_content(
        prompt,
        safety_settings={
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
        }
    )
    return response

def extract_final_classification(response_text):
    lines = response_text.split('\n')
    for line in lines:
        if 'Final Classification:' in line:
            answer = line.split(':')[1].strip()
            # sin puntuacion y lowercase:
            answer = re.sub(r'[^\w\s]', '', answer.lower())
            return answer
    return None

In [ ]:
responses = []
for review in tqdm(examples['text']):
    response = cot_clf(review)
    responses.append(response)

100%|██████████| 5/5 [00:08<00:00,  1.67s/it]


In [ ]:
def print_results(examples, responses):
    for example, response in zip(examples, responses):
        print(f"Review: {example['text']}")
        try:
            print(f"Response: {response.text}")
        except Exception as e:
            print(f"Error: {e}")
            print(f"Safety ratings: {response.candidates[0].safety_ratings}")
        else:
            print(f"Final Classification: {extract_final_classification(response.text)}")
        print(f"Ground truth: {id2label[example['label']]}")
        print("-"*80)

print_results(examples, responses)

Review: it's like an old warner bros . costumer jived with sex -- this could be the movie errol flynn always wanted to make , though bette davis , cast as joan , would have killed him .
Response: Here's a breakdown of the review's sentiment:

1. **Positive References:** 
    * "old warner bros. costumer jived": This suggests a classic, stylish film, evoking a sense of nostalgia and potentially high production value. 
    * "errol flynn always wanted to make":  This implies a film with exciting, adventurous themes, likely appealing to a certain audience.

2. **Mixed/Ambiguous References:**
    * "sex": This could be interpreted positively, indicating a potentially provocative or exciting aspect of the film. However, it could also be perceived as a negative element depending on the context and reviewer's taste.
    * "though bette davis, cast as Joan, would have killed him": This is a complex statement.  While it suggests a strong female character, it also implies a potentially volatile 

## Clasificación few-shot

Suele ser muy útil pasar ejemplos de cómo se resuelve la tarea. En este caso elegimos los ejemplos aleatoriamente pero podemos hacerlo manualmente.

In [ ]:
from datasets import concatenate_datasets

def get_few_shot_examples(dataset, num_examples=4):
    examples_pos = dataset['train'].filter(lambda x: x['label'] == 1).shuffle(seed=36).select(range(num_examples // 2))
    examples_neg = dataset['train'].filter(lambda x: x['label'] == 0).shuffle(seed=36).select(range(num_examples // 2))
    examples = concatenate_datasets([examples_pos, examples_neg]).shuffle(seed=5)
    out = []
    for example in examples:
        label = id2label[example['label']]
        formatted_example = (
            f'''Review: "{example['text']}"'''
            f'''\nReasoning: This review is classified as {label} because...'''
            f'''\nFinal Classification: {label}'''
        )
        out.append(formatted_example)
    return out

few_shot_examples = get_few_shot_examples(dataset)
for example in few_shot_examples:
    print(example)
    print("-"*80)

Filter:   0%|          | 0/8530 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8530 [00:00<?, ? examples/s]

Review: "[a] shapeless blob of desperate entertainment ."
Reasoning: This review is classified as negative because...
Final Classification: negative
--------------------------------------------------------------------------------
Review: "films about loss , grief and recovery are pretty valuable these days . seen in that light , moonlight mile should strike a nerve in many ."
Reasoning: This review is classified as positive because...
Final Classification: positive
--------------------------------------------------------------------------------
Review: "it's super- violent , super-serious and super-stupid ."
Reasoning: This review is classified as negative because...
Final Classification: negative
--------------------------------------------------------------------------------
Review: "ruzowitzky has taken this mothball-y stuff and made a rather sturdy , old-fashioned entertainment out of it ."
Reasoning: This review is classified as positive because...
Final Classification: positive
-

In [ ]:
def cot_few_shot_prompt(review, few_shot_examples):
    prompt = (
        '''Classify the following movie review as "positive" or "negative",'''
        ''' and provide step-by-step reasoning for your classification.'''
        ''' At the end, summarize the classification in the format "Final Classification: [positive/negative]":\n\n'''
    )
    prompt += '\n\n'.join(few_shot_examples)
    prompt += f'\n\nReview: "{review}"'
    return prompt

print(cot_few_shot_prompt("EJEMPLO.", few_shot_examples))

Classify the following movie review as "positive" or "negative", and provide step-by-step reasoning for your classification. At the end, summarize the classification in the format "Final Classification: [positive/negative]":

Review: "[a] shapeless blob of desperate entertainment ."
Reasoning: This review is classified as negative because...
Final Classification: negative

Review: "films about loss , grief and recovery are pretty valuable these days . seen in that light , moonlight mile should strike a nerve in many ."
Reasoning: This review is classified as positive because...
Final Classification: positive

Review: "it's super- violent , super-serious and super-stupid ."
Reasoning: This review is classified as negative because...
Final Classification: negative

Review: "ruzowitzky has taken this mothball-y stuff and made a rather sturdy , old-fashioned entertainment out of it ."
Reasoning: This review is classified as positive because...
Final Classification: positive

Review: "EJEMP

In [ ]:
def cot_few_shot_clf(review, few_shot_examples):
    prompt = cot_few_shot_prompt(review, few_shot_examples)
    response = model.generate_content(
        prompt,
        safety_settings={
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
        }
    )
    return response

In [ ]:
responses = []
for review in tqdm(examples['text']):
    response = cot_few_shot_clf(review, few_shot_examples)
    responses.append(response)

100%|██████████| 5/5 [00:06<00:00,  1.27s/it]


In [ ]:
print_results(examples, responses)

Review: it's like an old warner bros . costumer jived with sex -- this could be the movie errol flynn always wanted to make , though bette davis , cast as joan , would have killed him .
Response: Reasoning: This review is classified as positive because, despite using a somewhat strange comparison to old Warner Bros. films, the reviewer ultimately suggests the film is a well-made, entertaining piece of "old-fashioned" filmmaking. The final statement suggests the film would have been a good fit for classic actors like Errol Flynn and Bette Davis, further implying the film has a certain quality and charm.

Final Classification: positive 

Final Classification: positive
Ground truth: positive
--------------------------------------------------------------------------------
Review: [lin chung's] voice is rather unexceptional , even irritating ( at least to this western ear ) , making it awfully hard to buy the impetus for the complicated love triangle that develops between the three central 

## Extracción de datos estructurados de texto libre

Veamos cómo extraer los ingredientes de una receta de manera programática.

In [ ]:
ds_recetas = load_dataset("somosnlp/RecetasDeLaAbuela", "version_1")

README.md:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

main.csv:   0%|          | 0.00/40.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20236 [00:00<?, ? examples/s]

In [ ]:
print(ds_recetas)

DatasetDict({
    train: Dataset({
        features: ['Id', 'Nombre', 'URL', 'Ingredientes', 'Pasos', 'Pais', 'Duracion', 'Categoria', 'Contexto', 'Valoracion y Votos', 'Comensales', 'Tiempo', 'Dificultad', 'Valor nutricional'],
        num_rows: 20236
    })
})


In [ ]:
ds_small = ds_recetas["train"].filter(lambda x: x["Pais"] == "ARG").shuffle(seed=33).select(range(5))

Filter:   0%|          | 0/20236 [00:00<?, ? examples/s]

In [ ]:
# A veces los textos son listas no parseadas como tales.
# En tal caso, hacemos un join de la lista.
import re

def preprocess(example):
    """
    """
    if example["Pasos"].startswith("["):
        pasos_list = eval(example["Pasos"].encode('unicode_escape'))
        example["Pasos"] = " ".join(pasos_list)
    # Eliminamos whitespace duplicado:
    example["Pasos"] = re.sub(r'\s+', ' ', example["Pasos"])
    return example

ds_small = ds_small.map(preprocess)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
for example in ds_small:
    print(example["Ingredientes"])
    print(example["Pasos"])
    print("-"*80)

['1 paquete de galletitas de chocolate (230 gramos)', '400 gramos de dulce de leche tradicional', '400 gramos de queso untable', '100 centímetros cúbicos de leche', '1 cucharada postre de café instantáneo', '1 paquete de galletitas rellenas (90 gramos)', '4 cucharadas de postre de dulce de leche', '100 gramos de frutillas']
1 El primer paso para hacer esta torta de chocotorta decorada consiste en mezclar el dulce de leche con el queso untable. Aquí combinamos la misma cantidad de uno y otro, pero puedes hacerlo con más queso o más dulce. También puedes elegir un queso descremado o entero. 2 Mezcla la leche con el café instantáneo. En él, humedecerás cada galletita antes de armar con ellas la base de la chocotorta. Puedes mezclar café con cacao dulce o reemplazarlo por completo si haces esta torta para chicos. Truco: el café resaltará el sabor del chocolate y bajará un poco el punto dulce de la torta. 3 Para armar la torta, elige un molde redondo, en este caso de 16 centímetros de diáme

In [ ]:
def prompt_extract_json(receta):
    prompt = (
        '''Extraer los ingredientes de la siguiente receta en formato JSON.'''
        f'''\nReceta: {receta}'''
    )
    return prompt

print(prompt_extract_json(ds_small[0]["Pasos"]))

Extraer los ingredientes de la siguiente receta en formato JSON.
Receta: 1 El primer paso para hacer esta torta de chocotorta decorada consiste en mezclar el dulce de leche con el queso untable. Aquí combinamos la misma cantidad de uno y otro, pero puedes hacerlo con más queso o más dulce. También puedes elegir un queso descremado o entero. 2 Mezcla la leche con el café instantáneo. En él, humedecerás cada galletita antes de armar con ellas la base de la chocotorta. Puedes mezclar café con cacao dulce o reemplazarlo por completo si haces esta torta para chicos. Truco: el café resaltará el sabor del chocolate y bajará un poco el punto dulce de la torta. 3 Para armar la torta, elige un molde redondo, en este caso de 16 centímetros de diámetro. Cúbrelo con papel film para que puedas desmoldar luego. Haz una capa base con galletitas húmedas y, luego, agrega el relleno de dulce con queso para hacer otra capa de galletitas hasta utilizar todo. La última capa debería ser de galletitas. 4 Llev

In [ ]:
def extract_json(receta):
    prompt = prompt_extract_json(receta)
    response = model.generate_content(
        prompt,
        generation_config={'response_mime_type':'application/json'}
    )
    return response

In [ ]:
responses = []
for receta in tqdm(ds_small['Pasos']):
    response = extract_json(receta)
    responses.append(response)

100%|██████████| 5/5 [00:04<00:00,  1.01it/s]


In [ ]:
for example, response in zip(ds_small, responses):
    print(f"Receta: {example['Pasos']}")
    try:
        print(f"Response: {response.text}")
    except Exception as e:
        print(f"Error: {e}")

Receta: 1 El primer paso para hacer esta torta de chocotorta decorada consiste en mezclar el dulce de leche con el queso untable. Aquí combinamos la misma cantidad de uno y otro, pero puedes hacerlo con más queso o más dulce. También puedes elegir un queso descremado o entero. 2 Mezcla la leche con el café instantáneo. En él, humedecerás cada galletita antes de armar con ellas la base de la chocotorta. Puedes mezclar café con cacao dulce o reemplazarlo por completo si haces esta torta para chicos. Truco: el café resaltará el sabor del chocolate y bajará un poco el punto dulce de la torta. 3 Para armar la torta, elige un molde redondo, en este caso de 16 centímetros de diámetro. Cúbrelo con papel film para que puedas desmoldar luego. Haz una capa base con galletitas húmedas y, luego, agrega el relleno de dulce con queso para hacer otra capa de galletitas hasta utilizar todo. La última capa debería ser de galletitas. 4 Lleva la chocotorta argentina a la heladera por al menos unos 60 minu

In [ ]:
def prompt_extract_json2(receta):
    prompt = (
        '''Extraer los ingredientes de la siguiente receta en formato JSON.'''
        ''' El JSON debe contener únicamente la clave "ingredientes".'''
        f'''\nReceta: {receta}'''
    )
    return prompt

def extract_json2(receta):
    prompt = prompt_extract_json2(receta)
    response = model.generate_content(
        prompt,
        generation_config={'response_mime_type':'application/json'}
    )
    return response

responses = []
for receta in tqdm(ds_small['Pasos']):
    response = extract_json2(receta)
    responses.append(response)

100%|██████████| 5/5 [00:04<00:00,  1.02it/s]


In [ ]:
results = []
for response in responses:
    try:
        data = json.loads(response.text)
        results.append(data)
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
results[0]

{'ingredientes': ['dulce de leche',
  'queso untable',
  'leche',
  'café instantáneo',
  'galletitas',
  'cacao dulce',
  'papel film']}

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,ingredientes
0,"[dulce de leche, queso untable, leche, café in..."
1,"[maíz, leche, azúcar, vainilla, canela en polvo]"
2,"[cebolla, aceite, manteca, agua caliente, vino..."
3,"[bondiola, aceite, vino, maicena, agua]"
4,"[carne, huevo, leche, pimienta, provenzal, mos..."


In [ ]:
df['ingredientes'].explode().value_counts()

,count
ingredientes,
leche,4
aceite,3
cebolla,2
vino,2
maicena,2
huevo,2
batata,1
semillas,1
panko,1


In [ ]:
def prompt_extract_json3(receta):
    prompt = (
        '''Extraer los ingredientes de la siguiente receta en formato JSON.'''
        ''' El JSON debe contener las claves "veganos" y "no_veganos".'''
        ''' Ambas claves deben contener una lista de ingredientes.'''
        ''' Si no hay ingredientes para alguna clave, se debe incluir una lista vacía.'''
        f'''\nReceta: {receta}'''
    )
    return prompt

def extract_json3(receta):
    prompt = prompt_extract_json3(receta)
    response = model.generate_content(
        prompt,
        generation_config={'response_mime_type':'application/json'}
    )
    return response

responses = []
for receta in tqdm(ds_small['Pasos']):
    response = extract_json3(receta)
    responses.append(response)

100%|██████████| 5/5 [00:04<00:00,  1.01it/s]


In [ ]:
results = []
for response in responses:
    try:
        data = json.loads(response.text)
        results.append(data)
    except Exception as e:
        print(f"Error: {e}")

df = pd.DataFrame(results)
df.head()

,veganos,no_veganos
0,[],"[dulce de leche, queso untable, leche, café in..."
1,"[maíz, leche, azúcar, vainilla, canela en polvo]",[]
2,[],"[cebolla, aceite, manteca, agua caliente, vino..."
3,[],"[bondiola, aceite, vino, maicena, agua]"
4,"[pan rallado, avena, polenta, copos de maíz pr...","[carne de pollo, carne de cerdo, carne de vaca..."


In [ ]:
df['veganos'].explode().value_counts()

,count
veganos,
maíz,1
semillas,1
condimentos,1
tomate,1
vino,1
cebolla,1
aceite,1
boniato,1
batata,1


## Conversaciones multi-turno

Finalmente, con la API podemos simular un chat con en la interfaz gráfica.

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')
chat = model.start_chat(history=[])
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={},
        safety_settings={},
        tools=None,
        system_instruction=None,
        cached_content=None
    ),
    history=[]
)

In [ ]:
response = chat.send_message("¿Por qué el agua es azul?")
to_markdown(response.text)

> El agua no es realmente azul. El agua pura es en realidad incolora. La razón por la que la vemos azul es por la manera en que absorbe y dispersa la luz. 
> 
> 
> 
> * **Absorción de luz:** El agua absorbe mejor la luz roja y amarilla, mientras que refleja mejor la luz azul y verde. 
> 
> * **Dispersión de luz:** Cuando la luz atraviesa el agua, las moléculas de agua dispersan la luz azul en todas direcciones. Esta dispersión de la luz azul es lo que hace que veamos el agua como azul.
> 
> 
> 
> Sin embargo, el color azul del agua se intensifica en:
> 
> 
> 
> * **Profundidades mayores:** Cuanto más profundo es el agua, más luz roja y amarilla se absorbe, y la luz azul se dispersa más, haciendo que el agua se vea más azul.
> 
> * **Cielos despejados:** El cielo azul refleja luz azul sobre el agua, intensificando su color.
> 
> 
> 
> En resumen, el agua no tiene un color propio, sino que refleja la luz de su entorno. La absorción y dispersión de la luz hacen que el agua se vea azul, especialmente en aguas profundas y bajo cielos despejados.
> 


In [ ]:
response = chat.send_message("¿Cómo se lo explicarás a un niño de 8 años?")
to_markdown(response.text)

> Imagina que la luz del sol es como un arcoíris con todos los colores. Cuando la luz del sol llega al agua, la  "agua bebe" algunos colores, como el rojo y el amarillo, y deja ir los colores azules y verdes. 
> 
> 
> 
> Es como si el agua tuviera un filtro especial que solo deja pasar los colores azules y verdes. ¡Por eso la vemos azul!  
> 
> 
> 
> Y si el agua es muy profunda, la luz tiene que atravesar mucha agua, y se "bebe" casi todos los colores, quedando solo el azul, ¡por eso el mar se ve tan azul! 
> 


In [ ]:
for message in chat.history:
    display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **user**: ¿Por qué el agua es azul?

> **model**: El agua no es realmente azul. El agua pura es en realidad incolora. La razón por la que la vemos azul es por la manera en que absorbe y dispersa la luz. 
> 
> 
> 
> * **Absorción de luz:** El agua absorbe mejor la luz roja y amarilla, mientras que refleja mejor la luz azul y verde. 
> 
> * **Dispersión de luz:** Cuando la luz atraviesa el agua, las moléculas de agua dispersan la luz azul en todas direcciones. Esta dispersión de la luz azul es lo que hace que veamos el agua como azul.
> 
> 
> 
> Sin embargo, el color azul del agua se intensifica en:
> 
> 
> 
> * **Profundidades mayores:** Cuanto más profundo es el agua, más luz roja y amarilla se absorbe, y la luz azul se dispersa más, haciendo que el agua se vea más azul.
> 
> * **Cielos despejados:** El cielo azul refleja luz azul sobre el agua, intensificando su color.
> 
> 
> 
> En resumen, el agua no tiene un color propio, sino que refleja la luz de su entorno. La absorción y dispersión de la luz hacen que el agua se vea azul, especialmente en aguas profundas y bajo cielos despejados.
> 


> **user**: ¿Cómo se lo explicarás a un niño de 8 años?

> **model**: Imagina que la luz del sol es como un arcoíris con todos los colores. Cuando la luz del sol llega al agua, la  "agua bebe" algunos colores, como el rojo y el amarillo, y deja ir los colores azules y verdes. 
> 
> 
> 
> Es como si el agua tuviera un filtro especial que solo deja pasar los colores azules y verdes. ¡Por eso la vemos azul!  
> 
> 
> 
> Y si el agua es muy profunda, la luz tiene que atravesar mucha agua, y se "bebe" casi todos los colores, quedando solo el azul, ¡por eso el mar se ve tan azul! 
> 
